In [1]:
# !pip install --quiet  datasets #to access squad dataset
# !pip install --quiet pyarrow   #to deal with parquet files for saving dataset if required
# !pip install --quiet  tqdm     #for progress bars
# !pip install --quiet transformers # for t5 model
# !pip install --quiet tokenizers  #tokenizers from HuggingFace
# !pip install --quiet sentencepiece #subword tokenizer used by T5
# !pip install --quiet pytorch-lightning # pytorch wrapper 
# !pip install --quiet torchtext # text utilities

# Contructed random 50% dataset

In [1]:
#imports
import pandas as pd
import torch
from tqdm import tqdm
from datasets import DatasetDict, Dataset, load_from_disk
# from torch.utils.data import Dataset, DataLoader
from pprint import pprint
import copy
import numpy as np
from collections import defaultdict
import ipdb, re

pd.options.display.max_rows , pd.options.display.max_columns  = 100,100  

# Fraction for sample dataframe
proportion = 0.5

In [2]:
# DOCTEAT


# AUGMENTED
train_f1_data_path = "../data/df_train_tdms_augmented_summarized_with_id_f1_all_templates.parquet"
dev_f1_data_path = "../data/df_dev_tdms_augmented_summarized_with_id_f1_all_templates.parquet"
zeroshot_f1_data_path = "../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_all_templates.parquet"

train_f2_data_path = "../data/df_train_tdms_augmented_summarized_with_id_f2_all_templates.parquet"
dev_f2_data_path = "../data/df_dev_tdms_augmented_summarized_with_id_f2_all_templates.parquet"


df_train_tdms_f1 = pd.read_parquet(train_f1_data_path)
df_dev_tdms_f1 = pd.read_parquet(dev_f1_data_path)
df_zeroshot_tdms_f1 = pd.read_parquet(zeroshot_f1_data_path)

df_train_tdms_f2 = pd.read_parquet(train_f2_data_path)
df_dev_tdms_f2 = pd.read_parquet(dev_f2_data_path)

df_train_tdms_f1.head()

,id,prompt,answer
0,1707.03497v2.pdf,Title:\tValue Prediction Network\n\nAbstract:\...,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data..."
1,1707.03497v2.pdf,Read this and answer the question. If the ques...,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data..."
2,1707.03497v2.pdf,Title:\tValue Prediction Network\n\nAbstract:\...,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data..."
3,1707.03497v2.pdf,Title:\tValue Prediction Network\n\nAbstract:\...,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data..."
4,1707.03497v2.pdf,Title:\tValue Prediction Network\n\nAbstract:\...,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data..."


In [3]:
# substring_to_match_squad_1 = re.escape('Please answer a question about this article. If the question is unanswerable, say "unanswerable".')
# substring_to_match_squad_1

In [4]:
# Filter rows where the 'text' column contains the given substring
# substring_to_match_squad_1 = re.escape('Please answer a question about this article. If the question is unanswerable, say "unanswerable".')
substring_to_match_squad_1 = 'Please answer a question about this article. If the question is unanswerable, say "unanswerable".'
substring_to_match_squad_2 = 'Read this and answer the question. If the question is unanswerable, say \"unanswerable\".'
substring_to_match_squad_3 = '(If the question is unanswerable, say \"unanswerable\"'
substring_to_match_squad_4 = 'Try to answer this question if possible (otherwise reply \"unanswerable\")'
substring_to_match_squad_5 = 'If it is possible to answer this question, answer it for me (else, reply \"unanswerable\"):'
substring_to_match_squad_6 = 'Answer this question, if possible (if impossible, reply \"unanswerable\"):'
substring_to_match_squad_7 = 'What is the answer? (If it cannot be answered, return \"unanswerable\")'
substring_to_match_squad_8 = 'Now answer this question, if there is an answer (If it cannot be answered, return \"unanswerable\"):'

substring_to_match_drop_1 = 'Answer based on context:'
substring_to_match_drop_2 = 'Answer this question based on the article:'
# substring_to_match_drop_3 = ''
substring_to_match_drop_4 = 'Answer this question: '
substring_to_match_drop_5 = 'Read this article and answer this question'
substring_to_match_drop_6 = 'Based on the above article, answer a question.'
substring_to_match_drop_7 = 'Context: '

# TDMS
df_train_tdms_f1['template'] = df_train_tdms_f1['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )
df_dev_tdms_f1['template'] = df_dev_tdms_f1['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )

df_zeroshot_tdms_f1['template'] = df_zeroshot_tdms_f1['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )

df_train_tdms_f2['template'] = df_train_tdms_f2['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )
df_dev_tdms_f2['template'] = df_dev_tdms_f2['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )

df_zeroshot_tdms_f1.describe()

,id,prompt,answer,template
count,18000,18000,18000,18000
unique,1200,17955,653,15
top,2209.03182v1.pdf,Context: Title:\t\n\nAbstract:\t\n\n[image]\n\...,unanswerable,drop_7
freq,15,3,8220,1202


In [5]:
df_train_tdms_f1_squad_1 = df_train_tdms_f1[df_train_tdms_f1['template']=='squad_1'].sample(frac=proportion)
df_train_tdms_f1_squad_2 = df_train_tdms_f1[df_train_tdms_f1['template']=='squad_2'].sample(frac=proportion)
df_train_tdms_f1_squad_3 = df_train_tdms_f1[df_train_tdms_f1['template']=='squad_3'].sample(frac=proportion)
df_train_tdms_f1_squad_4 = df_train_tdms_f1[df_train_tdms_f1['template']=='squad_4'].sample(frac=proportion)
df_train_tdms_f1_squad_5 = df_train_tdms_f1[df_train_tdms_f1['template']=='squad_5'].sample(frac=proportion)
df_train_tdms_f1_squad_6 = df_train_tdms_f1[df_train_tdms_f1['template']=='squad_6'].sample(frac=proportion)
df_train_tdms_f1_squad_7 = df_train_tdms_f1[df_train_tdms_f1['template']=='squad_7'].sample(frac=proportion)
df_train_tdms_f1_squad_8 = df_train_tdms_f1[df_train_tdms_f1['template']=='squad_8'].sample(frac=proportion)

df_train_tdms_f1_drop_1 = df_train_tdms_f1[df_train_tdms_f1['template']=='drop_1'].sample(frac=proportion)
df_train_tdms_f1_drop_2 = df_train_tdms_f1[df_train_tdms_f1['template']=='drop_2'].sample(frac=proportion)
df_train_tdms_f1_drop_3 = df_train_tdms_f1[df_train_tdms_f1['template']=='drop_3'].sample(frac=proportion)
df_train_tdms_f1_drop_4 = df_train_tdms_f1[df_train_tdms_f1['template']=='drop_4'].sample(frac=proportion)
df_train_tdms_f1_drop_5 = df_train_tdms_f1[df_train_tdms_f1['template']=='drop_5'].sample(frac=proportion)
df_train_tdms_f1_drop_6 = df_train_tdms_f1[df_train_tdms_f1['template']=='drop_6'].sample(frac=proportion)
df_train_tdms_f1_drop_7 = df_train_tdms_f1[df_train_tdms_f1['template']=='drop_7'].sample(frac=proportion)

df_train_tdms_f1_50_percent = pd.concat([
    df_train_tdms_f1_squad_1, 
    df_train_tdms_f1_squad_2, 
    df_train_tdms_f1_squad_3, 
    df_train_tdms_f1_squad_4, 
    df_train_tdms_f1_squad_5, 
    df_train_tdms_f1_squad_6, 
    df_train_tdms_f1_squad_7, 
    df_train_tdms_f1_squad_8, 
    df_train_tdms_f1_drop_1, 
    df_train_tdms_f1_drop_2, 
    df_train_tdms_f1_drop_3, 
    df_train_tdms_f1_drop_4, 
    df_train_tdms_f1_drop_5,
    df_train_tdms_f1_drop_6,
    df_train_tdms_f1_drop_7
    ], ignore_index=True)
print("df_train_tdms_f1_50_percent")
display(df_train_tdms_f1_50_percent.describe())


df_dev_tdms_f1_squad_1 = df_dev_tdms_f1[df_dev_tdms_f1['template']=='squad_1'].sample(frac=proportion)
df_dev_tdms_f1_squad_2 = df_dev_tdms_f1[df_dev_tdms_f1['template']=='squad_2'].sample(frac=proportion)
df_dev_tdms_f1_squad_3 = df_dev_tdms_f1[df_dev_tdms_f1['template']=='squad_3'].sample(frac=proportion)
df_dev_tdms_f1_squad_4 = df_dev_tdms_f1[df_dev_tdms_f1['template']=='squad_4'].sample(frac=proportion)
df_dev_tdms_f1_squad_5 = df_dev_tdms_f1[df_dev_tdms_f1['template']=='squad_5'].sample(frac=proportion)
df_dev_tdms_f1_squad_6 = df_dev_tdms_f1[df_dev_tdms_f1['template']=='squad_6'].sample(frac=proportion)
df_dev_tdms_f1_squad_7 = df_dev_tdms_f1[df_dev_tdms_f1['template']=='squad_7'].sample(frac=proportion)
df_dev_tdms_f1_squad_8 = df_dev_tdms_f1[df_dev_tdms_f1['template']=='squad_8'].sample(frac=proportion)

df_dev_tdms_f1_drop_1 = df_dev_tdms_f1[df_dev_tdms_f1['template']=='drop_1'].sample(frac=proportion)
df_dev_tdms_f1_drop_2 = df_dev_tdms_f1[df_dev_tdms_f1['template']=='drop_2'].sample(frac=proportion)
df_dev_tdms_f1_drop_3 = df_dev_tdms_f1[df_dev_tdms_f1['template']=='drop_3'].sample(frac=proportion)
df_dev_tdms_f1_drop_4 = df_dev_tdms_f1[df_dev_tdms_f1['template']=='drop_4'].sample(frac=proportion)
df_dev_tdms_f1_drop_5 = df_dev_tdms_f1[df_dev_tdms_f1['template']=='drop_5'].sample(frac=proportion)
df_dev_tdms_f1_drop_6 = df_dev_tdms_f1[df_dev_tdms_f1['template']=='drop_6'].sample(frac=proportion)
df_dev_tdms_f1_drop_7 = df_dev_tdms_f1[df_dev_tdms_f1['template']=='drop_7'].sample(frac=proportion)

df_dev_tdms_f1_50_percent = pd.concat([
    df_dev_tdms_f1_squad_1, 
    df_dev_tdms_f1_squad_2, 
    df_dev_tdms_f1_squad_3, 
    df_dev_tdms_f1_squad_4, 
    df_dev_tdms_f1_squad_5, 
    df_dev_tdms_f1_squad_6, 
    df_dev_tdms_f1_squad_7, 
    df_dev_tdms_f1_squad_8, 
    df_dev_tdms_f1_drop_1, 
    df_dev_tdms_f1_drop_2, 
    df_dev_tdms_f1_drop_3, 
    df_dev_tdms_f1_drop_4, 
    df_dev_tdms_f1_drop_5,
    df_dev_tdms_f1_drop_6,
    df_dev_tdms_f1_drop_7
    ], ignore_index=True)
print("df_dev_tdms_f1_50_percent")
display(df_dev_tdms_f1_50_percent.describe())

df_zeroshot_tdms_f1_squad_1 = df_zeroshot_tdms_f1[df_zeroshot_tdms_f1['template']=='squad_1'].sample(frac=proportion)
df_zeroshot_tdms_f1_squad_2 = df_zeroshot_tdms_f1[df_zeroshot_tdms_f1['template']=='squad_2'].sample(frac=proportion)
df_zeroshot_tdms_f1_squad_3 = df_zeroshot_tdms_f1[df_zeroshot_tdms_f1['template']=='squad_3'].sample(frac=proportion)
df_zeroshot_tdms_f1_squad_4 = df_zeroshot_tdms_f1[df_zeroshot_tdms_f1['template']=='squad_4'].sample(frac=proportion)
df_zeroshot_tdms_f1_squad_5 = df_zeroshot_tdms_f1[df_zeroshot_tdms_f1['template']=='squad_5'].sample(frac=proportion)
df_zeroshot_tdms_f1_squad_6 = df_zeroshot_tdms_f1[df_zeroshot_tdms_f1['template']=='squad_6'].sample(frac=proportion)
df_zeroshot_tdms_f1_squad_7 = df_zeroshot_tdms_f1[df_zeroshot_tdms_f1['template']=='squad_7'].sample(frac=proportion)
df_zeroshot_tdms_f1_squad_8 = df_zeroshot_tdms_f1[df_zeroshot_tdms_f1['template']=='squad_8'].sample(frac=proportion)

df_zeroshot_tdms_f1_drop_1 = df_zeroshot_tdms_f1[df_zeroshot_tdms_f1['template']=='drop_1'].sample(frac=proportion)
df_zeroshot_tdms_f1_drop_2 = df_zeroshot_tdms_f1[df_zeroshot_tdms_f1['template']=='drop_2'].sample(frac=proportion)
df_zeroshot_tdms_f1_drop_3 = df_zeroshot_tdms_f1[df_zeroshot_tdms_f1['template']=='drop_3'].sample(frac=proportion)
df_zeroshot_tdms_f1_drop_4 = df_zeroshot_tdms_f1[df_zeroshot_tdms_f1['template']=='drop_4'].sample(frac=proportion)
df_zeroshot_tdms_f1_drop_5 = df_zeroshot_tdms_f1[df_zeroshot_tdms_f1['template']=='drop_5'].sample(frac=proportion)
df_zeroshot_tdms_f1_drop_6 = df_zeroshot_tdms_f1[df_zeroshot_tdms_f1['template']=='drop_6'].sample(frac=proportion)
df_zeroshot_tdms_f1_drop_7 = df_zeroshot_tdms_f1[df_zeroshot_tdms_f1['template']=='drop_7'].sample(frac=proportion)

df_zeroshot_tdms_f1_50_percent = pd.concat([
    df_zeroshot_tdms_f1_squad_1, 
    df_zeroshot_tdms_f1_squad_2, 
    df_zeroshot_tdms_f1_squad_3, 
    df_zeroshot_tdms_f1_squad_4, 
    df_zeroshot_tdms_f1_squad_5, 
    df_zeroshot_tdms_f1_squad_6, 
    df_zeroshot_tdms_f1_squad_7, 
    df_zeroshot_tdms_f1_squad_8, 
    df_zeroshot_tdms_f1_drop_1, 
    df_zeroshot_tdms_f1_drop_2, 
    df_zeroshot_tdms_f1_drop_3, 
    df_zeroshot_tdms_f1_drop_4, 
    df_zeroshot_tdms_f1_drop_5,
    df_zeroshot_tdms_f1_drop_6,
    df_zeroshot_tdms_f1_drop_7
    ], ignore_index=True)
print("df_zeroshot_tdms_f1_50_percent")
display(df_zeroshot_tdms_f1_50_percent.describe())

df_train_tdms_f2_squad_1 = df_train_tdms_f2[df_train_tdms_f2['template']=='squad_1'].sample(frac=proportion)
df_train_tdms_f2_squad_2 = df_train_tdms_f2[df_train_tdms_f2['template']=='squad_2'].sample(frac=proportion)
df_train_tdms_f2_squad_3 = df_train_tdms_f2[df_train_tdms_f2['template']=='squad_3'].sample(frac=proportion)
df_train_tdms_f2_squad_4 = df_train_tdms_f2[df_train_tdms_f2['template']=='squad_4'].sample(frac=proportion)
df_train_tdms_f2_squad_5 = df_train_tdms_f2[df_train_tdms_f2['template']=='squad_5'].sample(frac=proportion)
df_train_tdms_f2_squad_6 = df_train_tdms_f2[df_train_tdms_f2['template']=='squad_6'].sample(frac=proportion)
df_train_tdms_f2_squad_7 = df_train_tdms_f2[df_train_tdms_f2['template']=='squad_7'].sample(frac=proportion)
df_train_tdms_f2_squad_8 = df_train_tdms_f2[df_train_tdms_f2['template']=='squad_8'].sample(frac=proportion)

df_train_tdms_f2_drop_1 = df_train_tdms_f2[df_train_tdms_f2['template']=='drop_1'].sample(frac=proportion)
df_train_tdms_f2_drop_2 = df_train_tdms_f2[df_train_tdms_f2['template']=='drop_2'].sample(frac=proportion)
df_train_tdms_f2_drop_3 = df_train_tdms_f2[df_train_tdms_f2['template']=='drop_3'].sample(frac=proportion)
df_train_tdms_f2_drop_4 = df_train_tdms_f2[df_train_tdms_f2['template']=='drop_4'].sample(frac=proportion)
df_train_tdms_f2_drop_5 = df_train_tdms_f2[df_train_tdms_f2['template']=='drop_5'].sample(frac=proportion)
df_train_tdms_f2_drop_6 = df_train_tdms_f2[df_train_tdms_f2['template']=='drop_6'].sample(frac=proportion)
df_train_tdms_f2_drop_7 = df_train_tdms_f2[df_train_tdms_f2['template']=='drop_7'].sample(frac=proportion)

df_train_tdms_f2_50_percent = pd.concat([
    df_train_tdms_f2_squad_1, 
    df_train_tdms_f2_squad_2, 
    df_train_tdms_f2_squad_3, 
    df_train_tdms_f2_squad_4, 
    df_train_tdms_f2_squad_5, 
    df_train_tdms_f2_squad_6, 
    df_train_tdms_f2_squad_7, 
    df_train_tdms_f2_squad_8, 
    df_train_tdms_f2_drop_1, 
    df_train_tdms_f2_drop_2, 
    df_train_tdms_f2_drop_3, 
    df_train_tdms_f2_drop_4, 
    df_train_tdms_f2_drop_5,
    df_train_tdms_f2_drop_6,
    df_train_tdms_f2_drop_7
    ], ignore_index=True)
print("df_train_tdms_f2_50_percent")
display(df_train_tdms_f2_50_percent.describe())

df_dev_tdms_f2_squad_1 = df_dev_tdms_f2[df_dev_tdms_f2['template']=='squad_1'].sample(frac=proportion)
df_dev_tdms_f2_squad_2 = df_dev_tdms_f2[df_dev_tdms_f2['template']=='squad_2'].sample(frac=proportion)
df_dev_tdms_f2_squad_3 = df_dev_tdms_f2[df_dev_tdms_f2['template']=='squad_3'].sample(frac=proportion)
df_dev_tdms_f2_squad_4 = df_dev_tdms_f2[df_dev_tdms_f2['template']=='squad_4'].sample(frac=proportion)
df_dev_tdms_f2_squad_5 = df_dev_tdms_f2[df_dev_tdms_f2['template']=='squad_5'].sample(frac=proportion)
df_dev_tdms_f2_squad_6 = df_dev_tdms_f2[df_dev_tdms_f2['template']=='squad_6'].sample(frac=proportion)
df_dev_tdms_f2_squad_7 = df_dev_tdms_f2[df_dev_tdms_f2['template']=='squad_7'].sample(frac=proportion)
df_dev_tdms_f2_squad_8 = df_dev_tdms_f2[df_dev_tdms_f2['template']=='squad_8'].sample(frac=proportion)

df_dev_tdms_f2_drop_1 = df_dev_tdms_f2[df_dev_tdms_f2['template']=='drop_1'].sample(frac=proportion)
df_dev_tdms_f2_drop_2 = df_dev_tdms_f2[df_dev_tdms_f2['template']=='drop_2'].sample(frac=proportion)
df_dev_tdms_f2_drop_3 = df_dev_tdms_f2[df_dev_tdms_f2['template']=='drop_3'].sample(frac=proportion)
df_dev_tdms_f2_drop_4 = df_dev_tdms_f2[df_dev_tdms_f2['template']=='drop_4'].sample(frac=proportion)
df_dev_tdms_f2_drop_5 = df_dev_tdms_f2[df_dev_tdms_f2['template']=='drop_5'].sample(frac=proportion)
df_dev_tdms_f2_drop_6 = df_dev_tdms_f2[df_dev_tdms_f2['template']=='drop_6'].sample(frac=proportion)
df_dev_tdms_f2_drop_7 = df_dev_tdms_f2[df_dev_tdms_f2['template']=='drop_7'].sample(frac=proportion)

df_dev_tdms_f2_50_percent = pd.concat([
    df_dev_tdms_f2_squad_1, 
    df_dev_tdms_f2_squad_2, 
    df_dev_tdms_f2_squad_3, 
    df_dev_tdms_f2_squad_4, 
    df_dev_tdms_f2_squad_5, 
    df_dev_tdms_f2_squad_6, 
    df_dev_tdms_f2_squad_7, 
    df_dev_tdms_f2_squad_8, 
    df_dev_tdms_f2_drop_1, 
    df_dev_tdms_f2_drop_2, 
    df_dev_tdms_f2_drop_3, 
    df_dev_tdms_f2_drop_4, 
    df_dev_tdms_f2_drop_5,
    df_dev_tdms_f2_drop_6,
    df_dev_tdms_f2_drop_7
    ], ignore_index=True)
print("df_dev_tdms_f2_50_percent")
display(df_dev_tdms_f2_50_percent.describe())

df_train_tdms_f1_50_percent


,id,prompt,answer,template
count,92910,92910,92910,92910
unique,12388,92545,7976,15
top,1704.06326v2.pdf,Title:\t\n\nAbstract:\t\n\n[image]\n\nTry to a...,unanswerable,drop_7
freq,14,17,33072,6198


df_dev_tdms_f1_50_percent


,id,prompt,answer,template
count,12315,12315,12315,12315
unique,1642,12314,994,15
top,1101.0552.pdf,Title:\tA Sample ACM SIG Proceedings Paper in ...,unanswerable,drop_7
freq,14,2,4902,822


df_zeroshot_tdms_f1_50_percent


,id,prompt,answer,template
count,9000,9000,9000,9000
unique,1200,8984,653,15
top,1812.01936v1.pdf,Read this and answer the question. If the ques...,unanswerable,drop_7
freq,14,3,4122,601


df_train_tdms_f2_50_percent


,id,prompt,answer,template
count,92864,92864,92864,92864
unique,12380,92514,7987,15
top,1406.3123.pdf,Title:\t\n\nAbstract:\t\n\n[image]\n\nTry to a...,unanswerable,drop_7
freq,14,12,32682,6196


df_dev_tdms_f2_50_percent


,id,prompt,answer,template
count,12360,12360,12360,12360
unique,1648,12352,983,15
top,1410.3507.pdf,Title:\tTFNet: Multi-Semantic Feature Interact...,unanswerable,drop_7
freq,14,2,4977,825


In [6]:
(41340+41341)/2

41340.5

In [7]:
(17641+17640)/2

17640.5

In [8]:
df_train_tdms_f1_50_percent.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_50_percent.parquet')
df_dev_tdms_f1_50_percent.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_50_percent.parquet')
df_zeroshot_tdms_f1_50_percent.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_50_percent.parquet')

df_train_tdms_f2_50_percent.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_50_percent.parquet')
df_dev_tdms_f2_50_percent.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_50_percent.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_50_percent.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_50_percent.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_50_percent.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_50_percent.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_50_percent.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_50_PERCENT")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-cf3168d80144d7d1/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

Saving the dataset (0/3 shards):   0%|          | 0/92910 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/12315 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9000 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/92864 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/12360 [00:00<?, ? examples/s]